# Inferring graphs from surveys

In this notebook we discuss a simple method for representing survey data using graphs. This is where the [_surveygraph_](https://github.com/surveygraph) package comes in.

We assume users have a basic knowledge of R and graphs. In particular, we will be working with the _igraph_ package in order to visualise and analyse the graph data produced by _surveygraph_. An introduction to R and _igraph_ can be found on the _surveygraph_ [Github page](https://github.com/surveygraph/surveygraph-tutorials).

## Similarity measure

## Model survey data

## Agent projection

## Symbolic projection

## Thresholding respondent similarity

In [28]:
# Define custom CSS to narrow the width of the notebook container
custom_css <- "
<style>
  .container {
    max-width: 80%; /* Adjust this value to set the desired width */
    margin: 0 auto; /* Center the container */
  }
</style>
"

# Display the custom CSS
IRdisplay::display_html(custom_css)
